In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# Load the dataset
df = pd.read_csv('./data/heart_disease_dataset.csv')

# Display the first few rows of the dataframe
print(df.head())

# Preprocess the data
# Encode categorical variables
label_encoders = {}
for column in ['Gender', 'Obesity']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Remove rows with missing values
df = df.dropna()

# Features and target
X = df[['Age', 'Gender', 'Obesity', 'Heart Rate']]
y = df['Heart Disease']

# Check class distribution
print("Class distribution before SMOTE:")
print(y.value_counts())

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check class distribution after SMOTE
print("Class distribution after SMOTE:")
print(y_resampled.value_counts())

# Standardize the features
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Create and train the logistic regression model with hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [100, 200, 300]
}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model
model = grid_search.best_estimator_

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Function to predict heart attack based on input features
def predict_heart_attack(age, gender, obesity, heart_rate):
    gender = label_encoders['Gender'].transform([gender])[0]
    obesity = label_encoders['Obesity'].transform(['Yes' if obesity else 'No'])[0]
    features = scaler.transform([[age, gender, obesity, heart_rate]])
    prediction = model.predict(features)
    return 'Heart Attack' if prediction[0] == 1 else 'No Heart Attack'


   Age  Gender  Cholesterol  Blood Pressure  Heart Rate  Smoking  \
0   75  Female          228             119          66  Current   
1   48    Male          204             165          62  Current   
2   53    Male          234              91          67    Never   
3   69  Female          192              90          72  Current   
4   62  Female          172             163          93    Never   

  Alcohol Intake  Exercise Hours Family History Diabetes Obesity  \
0          Heavy               1             No       No     Yes   
1            NaN               5             No       No      No   
2          Heavy               3            Yes       No     Yes   
3            NaN               4             No      Yes      No   
4            NaN               6             No      Yes      No   

   Stress Level  Blood Sugar Exercise Induced Angina   Chest Pain Type  \
0             8          119                     Yes   Atypical Angina   
1             9           70      

In [98]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

In [99]:
print(predict_heart_attack(50, 'Male', True, 200)) 

No Heart Attack


In [100]:
def get_current_bpm_from_file():
    try:
        with open('current_bpm.txt', 'r') as f:
            bpm_str = f.read().strip()
            return int(bpm_str)
    except FileNotFoundError:
        print("File not found.")
        return None
    except ValueError:
        print("Value error.")
        return None
    
print(get_current_bpm_from_file())

186


In [101]:
import stream_bpm as bpm

def get_attack_status(age=34,gender='Male',obesity=True):
     bpm1 = get_current_bpm_from_file()
     if bpm1 is not None:
          # print("Hello")
          status= predict_heart_attack(age,gender,obesity,bpm1)
          print(status)
          return status
     
     
     

In [103]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
# import stream_bpm as bpm
import time

# Email configuration
gmail_user = 'vijayakumarmamadapur@gmail.com'
# gmail_password = 'Vijay6360237672'
gmail_password = 'ducs vllg bzcc trix'
to_email = 'vijaymamadapur143@gmail.com'

def send_email(subject, body, to_email):
    msg = MIMEMultipart()
    msg['From'] = gmail_user
    msg['To'] = to_email
    msg['Subject'] = subject
    
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(gmail_user, gmail_password)
        text = msg.as_string()
        server.sendmail(gmail_user, to_email, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

def monitor_heart_attack(age=34, gender='Male', obesity=True):
    consecutive_heart_attack_count = 0
    while True:
        status = get_attack_status(age, gender, obesity)
        # print(consecutive_heart_attack_count)
        if status == 'Heart Attack':
            consecutive_heart_attack_count += 1
        else:
            consecutive_heart_attack_count = 0
        
        if consecutive_heart_attack_count >= 10: #reduce this for count
            send_email("Heart Attack Alert", "Heart attack detected ten times consecutively!", to_email)
            consecutive_heart_attack_count = 0  # Reset counter after sending email
        
        # Add a delay between checks to avoid continuous execution (e.g., 10 seconds)
        time.sleep(6)

# send_email('Heart Attack found','Hey your relative has health issues! Try contacting him ASAP',to_email=to_email)
monitor_heart_attack(20,'Male',False)


No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0
No Heart Attack
0


KeyboardInterrupt: 

<h1>Non useful code #redundant</h1>

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report

# # Input data files are available in the read-only "../input/" directory
# import os

# # for dirname, _, filenames in os.walk('/kaggle/input'):
# #     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# # Load the dataset
# df = pd.read_csv('./data/heart_disease_dataset.csv')

# # Display the first few rows of the dataframe
# print(df.head())

# # Preprocess the data
# df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
# df['Obesity'] = df['Obesity'].map({'Yes': 1, 'No': 0})

# # Remove rows with missing values
# df = df.dropna()

# # Features and target
# X = df[['Age', 'Gender', 'Obesity', 'Heart Rate']]
# y = df['Heart Disease']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create and train the logistic regression model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy * 100:.2f}%")
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

# # Function to predict heart attack based on input features
# def predict_heart_attack(age, gender, obesity, heart_rate):
#     gender = 1 if gender == 'male' else 0
#     obesity = 1 if obesity else 0
#     prediction = model.predict([[age, gender, obesity, heart_rate]])
#     return 'Heart Attack' if prediction[0] == 1 else 'No Heart Attack'

# # Example prediction
# print(predict_heart_attack(50, 'male', True, 90))  # Replace with actual values

In [ ]:
# print(predict_heart_attack(50, 'male', True, 4))  